<a href="https://www.kaggle.com/code/lalit7881/cybercrime-compliance-index-global?scriptVersionId=297610192" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr
COLORS = {
    'primary': '#0D47A1',
    'secondary': '#00C853',
    'accent': '#FFB300',
    'danger': '#FF5252',
    'background': '#F5F5F5',
    'text': '#263238'
}

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/global-cybercrime-regulation-index-194-countries/cybercrime_regulation_change_2020_2024.csv
/kaggle/input/global-cybercrime-regulation-index-194-countries/global_cybercrime_regulation_index_clean.csv


## Cybercrime Compliance Index (Global)

In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)


## Import dataset

In [4]:
df = pd.read_csv('/kaggle/input/global-cybercrime-regulation-index-194-countries/global_cybercrime_regulation_index_clean.csv')
df_change = pd.read_csv('/kaggle/input/global-cybercrime-regulation-index-194-countries/cybercrime_regulation_change_2020_2024.csv')

In [5]:
# Check first few rows
display(df.head())
display(df_change.head())

# Check columns and data types
print(df.info())
print(df_change.info())

# Check for missing values
print(df.isnull().sum())
print(df_change.isnull().sum())


,country_code,country_name,year,cybercrime_regulation_score,region,income_level
0,AFG,Afghanistan,2020,0.0000,Asia,Low Income
1,AFG,Afghanistan,2024,0.1047,Asia,Low Income
2,ALB,Albania,2020,0.9328,Europe,Upper Middle Income
3,ALB,Albania,2024,1.0000,Europe,Upper Middle Income
4,DZA,Algeria,2020,0.6215,Africa,Upper Middle Income


,country_code,country_name,score_2020,score_2024,score_change,percent_change,region,income_level
0,AFG,Afghanistan,0.0000,0.1047,0.1047,inf,Asia,Low Income
1,ALB,Albania,0.9328,1.0000,0.0672,7.2100,Europe,Upper Middle Income
2,DZA,Algeria,0.6215,0.9226,0.3010,48.4300,Africa,Upper Middle Income
3,AND,Andorra,0.3490,0.8151,0.4662,133.5800,Other,Not Classified
4,AGO,Angola,0.3477,0.7020,0.3543,101.8700,Africa,Lower Middle Income


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   country_code                 388 non-null    object 
 1   country_name                 388 non-null    object 
 2   year                         388 non-null    int64  
 3   cybercrime_regulation_score  388 non-null    float64
 4   region                       388 non-null    object 
 5   income_level                 388 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 18.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country_code    194 non-null    object 
 1   country_name    194 non-null    object 
 2   score_2020      194 non-null    float64
 3   score_2024      194 non-null  

In [6]:
print(f"📏 Main Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"📏 Change Analysis Dataset: {df_change.shape[0]} rows × {df_change.shape[1]} columns")


📏 Main Dataset Shape: 388 rows × 6 columns
📏 Change Analysis Dataset: 194 rows × 8 columns


In [7]:
print(f"\n📅 Years Covered: {sorted(df['year'].unique())}")
print(f"🌍 Number of Countries: {df['country_name'].nunique()}")



📅 Years Covered: [np.int64(2020), np.int64(2024)]
🌍 Number of Countries: 194


In [8]:
print(f"\n🗂️ Column Names:\n{list(df.columns)}")



🗂️ Column Names:
['country_code', 'country_name', 'year', 'cybercrime_regulation_score', 'region', 'income_level']


In [9]:
display(df.head(10))


,country_code,country_name,year,cybercrime_regulation_score,region,income_level
0,AFG,Afghanistan,2020,0.0000,Asia,Low Income
1,AFG,Afghanistan,2024,0.1047,Asia,Low Income
2,ALB,Albania,2020,0.9328,Europe,Upper Middle Income
3,ALB,Albania,2024,1.0000,Europe,Upper Middle Income
4,DZA,Algeria,2020,0.6215,Africa,Upper Middle Income
5,DZA,Algeria,2024,0.9226,Africa,Upper Middle Income
6,AND,Andorra,2020,0.3490,Other,Not Classified
7,AND,Andorra,2024,0.8151,Other,Not Classified
8,AGO,Angola,2020,0.3477,Africa,Lower Middle Income
9,AGO,Angola,2024,0.7020,Africa,Lower Middle Income


In [10]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   country_code                 388 non-null    object 
 1   country_name                 388 non-null    object 
 2   year                         388 non-null    int64  
 3   cybercrime_regulation_score  388 non-null    float64
 4   region                       388 non-null    object 
 5   income_level                 388 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 18.3+ KB
None


In [11]:
df.isnull().sum()
df_change.isnull().sum()


country_code      0
country_name      0
score_2020        0
score_2024        0
score_change      0
percent_change    4
region            0
income_level      0
dtype: int64

In [12]:
df.describe()


,year,cybercrime_regulation_score
count,388.0000,388.0000
mean,2022.0000,0.6726
std,2.0026,0.3463
min,2020.0000,0.0000
25%,2020.0000,0.3669
50%,2022.0000,0.7935
75%,2024.0000,1.0000
max,2024.0000,1.0000


In [13]:
df.duplicated().sum()

np.int64(0)

In [14]:
display(df.groupby('year')['cybercrime_regulation_score'].describe().round(4))


,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2020,194.0000,0.6038,0.3806,0.0000,0.2843,0.6466,1.0000,1.0000
2024,194.0000,0.7413,0.2935,0.0000,0.5523,0.8311,1.0000,1.0000


In [15]:
missing = df.isnull().sum()
print(f"Total Missing Values: {missing.sum()}")
print(f"✅ Dataset Completeness: {100 - (missing.sum()/df.size)*100:.2f}%")


Total Missing Values: 0
✅ Dataset Completeness: 100.00%


In [16]:
for col in df.columns:
    print(f"{col:35s}: {df[col].nunique():4d} unique values")


country_code                       :  194 unique values
country_name                       :  194 unique values
year                               :    2 unique values
cybercrime_regulation_score        :  159 unique values
region                             :    8 unique values
income_level                       :    5 unique values


In [17]:
df_2024 = df[df['year'] == 2024]
region_dist = df_2024.groupby('region').agg({
    'country_name': 'count',
    'cybercrime_regulation_score': ['mean', 'median', 'std']
}).round(4)


In [18]:
income_dist = df_2024.groupby('income_level').agg({
    'country_name': 'count',
    'cybercrime_regulation_score': ['mean', 'median', 'std']
}).round(4)


## EDA

In [19]:
fig = px.histogram(
    df,
    x='cybercrime_regulation_score',
    color='year',
    marginal='box',
    nbins=30,
    title='📊 Distribution of Cybercrime Regulation Scores (2020 vs 2024)',
    labels={'cybercrime_regulation_score': 'Regulation Score', 'count': 'Number of Countries'},
    color_discrete_sequence=['#FF5252', '#00C853'],
    opacity=0.7
)


In [20]:
fig.update_layout(
    plot_bgcolor='white',
    font=dict(size=12, family='Arial'),
    title_font_size=18,
    height=500
)


## 📌 INTERPRETATION:
• The distribution shows a bimodal pattern with peaks near 0.35 and 1.0

• Many countries have either very low or perfect regulation scores

• 2024 shows more countries achieving higher scores compared to 2020

• The middle range (0.4-0.8) has moderate representation


In [21]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('2020 Distribution', '2024 Distribution'),
    specs=[[{'type': 'histogram'}, {'type': 'histogram'}]]
)


In [22]:
df_2020 = df[df['year'] == 2020]
df_2024 = df[df['year'] == 2024]


In [23]:
fig.add_trace(
    go.Histogram(x=df_2020['cybercrime_regulation_score'], 
                 nbinsx=30, 
                 marker_color=COLORS['primary'],
                 name='2020',
                 opacity=0.7),
    row=1, col=1
)


In [24]:
region_stats = df_2024.groupby('region')['cybercrime_regulation_score'].agg(['mean', 'count']).reset_index()
region_stats = region_stats.sort_values('mean', ascending=False)


In [25]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=region_stats['region'],
    y=region_stats['mean'],
    text=region_stats['mean'].round(3),
    texttemplate='%{text:.3f}',
    textposition='outside',
    marker_color=[COLORS['primary'], COLORS['secondary'], COLORS['accent'], 
                  '#9C27B0', '#FF5722', '#00BCD4', '#4CAF50'][:len(region_stats)],
    hovertemplate='<b>%{x}</b><br>Average Score: %{y:.4f}<br>Countries: %{customdata}<extra></extra>',
    customdata=region_stats['count']
))


In [26]:
fig.update_layout(
    title={
        'text': '<b>Average Cybercrime Regulation Score by Region (2024)</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 22}
    },
    xaxis_title="<b>Region</b>",
    yaxis_title="<b>Average Score</b>",
    template='plotly_white',
    height=500,
    yaxis=dict(range=[0, 1.1]),
    hovermode='x'
)


In [27]:
print("🏆 Top Region: ", region_stats.iloc[0]['region'], 
      f"({region_stats.iloc[0]['mean']:.4f})")
print("📉 Lowest Region: ", region_stats.iloc[-1]['region'], 
      f"({region_stats.iloc[-1]['mean']:.4f})")


🏆 Top Region:  Europe (0.9709)
📉 Lowest Region:  Oceania (0.4215)


In [28]:
regions = df_2024['region'].unique()

In [29]:
# Define colors for regions
colors_map = {
    'Europe': COLORS['primary'],
    'Asia': COLORS['secondary'],
    'Africa': COLORS['accent'],
    'North America': '#9C27B0',
    'South America': '#FF5722',
    'Middle East': '#00BCD4',
    'Oceania': '#4CAF50',
    'Other': '#757575'
}

In [30]:
for region in sorted(regions):
    region_data = df_2024[df_2024['region'] == region]['cybercrime_regulation_score']
    fig.add_trace(go.Box(
        y=region_data,
        name=region,
        marker_color=colors_map.get(region, '#757575'),
        boxmean='sd'  # Show mean and standard deviation
    ))


In [31]:
fig.update_layout(
    title={
        'text': '<b>Regional Score Distribution with Statistical Measures (2024)</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    yaxis_title="<b>Cybercrime Regulation Score</b>",
    xaxis_title="<b>Region</b>",
    template='plotly_white',
    height=550,
    showlegend=False,
    yaxis=dict(range=[-0.05, 1.05])
)


In [32]:
# Define income order and filter
income_order = ['High Income', 'Upper Middle Income', 'Lower Middle Income', 'Low Income', 'Not Classified']
df_2024_filtered = df_2024[df_2024['income_level'].isin(income_order[:4])]

# Colors per income level
income_colors = {
    'High Income': COLORS['secondary'],
    'Upper Middle Income': COLORS['accent'],
    'Lower Middle Income': '#FF7043',
    'Low Income': COLORS['danger']
}

# Create violin plot
fig = go.Figure()

for income in income_order[:4]:
    income_data = df_2024_filtered[df_2024_filtered['income_level'] == income]['cybercrime_regulation_score']
    fig.add_trace(go.Violin(
        y=income_data,
        name=income,
        box_visible=True,
        meanline_visible=True,
        fillcolor=income_colors[income],
        opacity=0.6,
        x0=income
    ))

# Layout
fig.update_layout(
    title={
        'text': '<b>Cybercrime Regulation Score Distribution by Income Level (2024)</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    yaxis_title="<b>Regulation Score</b>",
    xaxis_title="<b>Income Level</b>",
    template='plotly_white',
    height=550,
    showlegend=False,
    yaxis=dict(range=[-0.05, 1.05])
)

fig.show()

# Statistical summary by income level
print("="*70)
print("STATISTICAL SUMMARY BY INCOME LEVEL (2024)")
print("="*70)
for income in income_order[:4]:
    data = df_2024_filtered[df_2024_filtered['income_level'] == income]['cybercrime_regulation_score']
    print(f"\n{income}:")
    print(f"  Mean: {data.mean():.4f} | Median: {data.median():.4f} | Std: {data.std():.4f}")
    print(f"  Min: {data.min():.4f} | Max: {data.max():.4f} | N: {len(data)}")


STATISTICAL SUMMARY BY INCOME LEVEL (2024)

High Income:
  Mean: 0.9581 | Median: 1.0000 | Std: 0.1078
  Min: 0.4749 | Max: 1.0000 | N: 48

Upper Middle Income:
  Mean: 0.7795 | Median: 0.9226 | Std: 0.2814
  Min: 0.0537 | Max: 1.0000 | N: 52

Lower Middle Income:
  Mean: 0.7154 | Median: 0.7626 | Std: 0.2753
  Min: 0.0000 | Max: 1.0000 | N: 46

Low Income:
  Mean: 0.4976 | Median: 0.4899 | Std: 0.3004
  Min: 0.0000 | Max: 1.0000 | N: 29


In [33]:
fig.update_layout(
    title={
        'text': '<b>Cybercrime Regulation Score Distribution by Income Level (2024)</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    yaxis_title="<b>Regulation Score</b>",
    xaxis_title="<b>Income Level</b>",
    template='plotly_white',
    height=550,
    showlegend=False,
    yaxis=dict(range=[-0.05, 1.05])
)


In [34]:
income_comparison = df.groupby(['year', 'income_level'])['cybercrime_regulation_score'].mean().reset_index()
income_comparison = income_comparison[income_comparison['income_level'].isin(income_order[:4])]


In [35]:
for year in [2020, 2024]:
    year_data = income_comparison[income_comparison['year'] == year]
    fig.add_trace(go.Bar(
        name=str(year),
        x=year_data['income_level'],
        y=year_data['cybercrime_regulation_score'],
        text=year_data['cybercrime_regulation_score'].round(3),
        texttemplate='%{text:.3f}',
        textposition='outside',
        marker_color=COLORS['danger'] if year == 2020 else COLORS['secondary']
    ))


In [36]:
fig.update_layout(
    title='<b>Income Level Comparison: 2020 vs 2024</b>',
    xaxis_title='<b>Income Level</b>',
    yaxis_title='<b>Average Score</b>',
    barmode='group',
    template='plotly_white',
    height=500,
    yaxis=dict(range=[0, 1.0])
)


In [37]:
scatter_data = df.pivot(index='country_name', columns='year', values='cybercrime_regulation_score').reset_index()
scatter_data = scatter_data.merge(df_2024[['country_name', 'region', 'income_level']], on='country_name')


In [38]:
fig = px.scatter(
    scatter_data,
    x=2020,
    y=2024,
    hover_name='country_name',
    color='region',
    size=abs(scatter_data[2024] - scatter_data[2020]),
    size_max=20,
    title='<b>Cybercrime Regulation Scores: 2020 vs 2024 Comparison</b>',
    labels={2020: '2020 Score', 2024: '2024 Score'},
    color_discrete_sequence=px.colors.qualitative.Bold
)


In [39]:
fig.add_trace(go.Scatter(
    x=[0, 1],
    y=[0, 1],
    mode='lines',
    name='No Change Line',
    line=dict(color='gray', dash='dash', width=2),
    showlegend=True
))


In [40]:
fig.update_layout(
    template='plotly_white',
    height=600,
    xaxis=dict(range=[-0.05, 1.05]),
    yaxis=dict(range=[-0.05, 1.05])
)


In [41]:
improved = len(scatter_data[scatter_data[2024] > scatter_data[2020]])
declined = len(scatter_data[scatter_data[2024] < scatter_data[2020]])
unchanged = len(scatter_data[scatter_data[2024] == scatter_data[2020]])

print(f"✅ Improved: {improved} countries ({improved/len(scatter_data)*100:.1f}%)")
print(f"❌ Declined: {declined} countries ({declined/len(scatter_data)*100:.1f}%)")
print(f"➖ Unchanged: {unchanged} countries ({unchanged/len(scatter_data)*100:.1f}%)")
print(f"📊 Average change: {(scatter_data[2024] - scatter_data[2020]).mean():.4f}")
print(f"📈 Largest improvement: {(scatter_data[2024] - scatter_data[2020]).max():.4f}")
print(f"📉 Largest decline: {(scatter_data[2024] - scatter_data[2020]).min():.4f}")


✅ Improved: 112 countries (57.7%)
❌ Declined: 21 countries (10.8%)
➖ Unchanged: 61 countries (31.4%)
📊 Average change: 0.1374
📈 Largest improvement: 0.8317
📉 Largest decline: -0.4552


In [42]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Top 10 Improvers', 'Top 10 Decliners'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)


In [43]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Example: COLORS dictionary
COLORS = {
    'primary': '#0D47A1',
    'secondary': '#00C853',
    'accent': '#FFB300',
    'danger': '#FF5252',
    'background': '#F5F5F5',
    'text': '#263238'
}

# Calculate score change
scatter_data['score_change'] = scatter_data[2024] - scatter_data[2020]

# Top 10 improvers and decliners
top_improvers = scatter_data.sort_values('score_change', ascending=False).head(10)
top_decliners = scatter_data.sort_values('score_change', ascending=True).head(10)

# Create subplot
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Top 10 Improvers', 'Top 10 Decliners'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

# Top improvers
fig.add_trace(
    go.Bar(
        y=top_improvers['country_name'][::-1],
        x=top_improvers['score_change'][::-1],
        orientation='h',
        marker_color=COLORS['secondary'],
        text=top_improvers['score_change'][::-1].round(3),
        texttemplate='%{text:.3f}',
        textposition='outside',
        name='Improvement'
    ),
    row=1, col=1
)

# Top decliners
fig.add_trace(
    go.Bar(
        y=top_decliners['country_name'][::-1],
        x=top_decliners['score_change'][::-1],
        orientation='h',
        marker_color=COLORS['danger'],
        text=top_decliners['score_change'][::-1].round(3),
        texttemplate='%{text:.3f}',
        textposition='outside',
        name='Decline'
    ),
    row=1, col=2
)

# Layout
fig.update_layout(
    title_text='<b>Countries with Largest Changes in Cybercrime Regulation (2020-2024)</b>',
    title_x=0.5,
    title_font_size=18,
    height=500,
    showlegend=False,
    template='plotly_white'
)

fig.update_xaxes(title_text="Score Change", row=1, col=1)
fig.update_xaxes(title_text="Score Change", row=1, col=2)

fig.show()


In [44]:
perfect_score_2024 = df_2024[df_2024['cybercrime_regulation_score'] == 1.0].copy()
perfect_score_2024 = perfect_score_2024.sort_values('country_name')


In [45]:
print(f"🌟 {len(perfect_score_2024)} COUNTRIES WITH PERFECT SCORE (1.0) IN 2024")


🌟 70 COUNTRIES WITH PERFECT SCORE (1.0) IN 2024


In [46]:
for region in perfect_score_2024['region'].unique():
    region_countries = perfect_score_2024[perfect_score_2024['region'] == region]['country_name'].values
    print(f"\n{region} ({len(region_countries)}):")
    for i, country in enumerate(region_countries, 1):
        print(f"  {i}. {country}")



Europe (33):
  1. Albania
  2. Belgium
  3. Bulgaria
  4. Croatia
  5. Cyprus
  6. Czechia
  7. Denmark
  8. Estonia
  9. Finland
  10. France
  11. Germany
  12. Greece
  13. Hungary
  14. Iceland
  15. Ireland
  16. Italy
  17. Latvia
  18. Luxembourg
  19. Malta
  20. Moldova
  21. Netherlands
  22. Norway
  23. Poland
  24. Portugal
  25. Romania
  26. Russian Federation
  27. Serbia
  28. Slovak Republic
  29. Slovenia
  30. Spain
  31. Sweden
  32. Switzerland
  33. United Kingdom

Oceania (1):
  1. Australia

Asia (14):
  1. Azerbaijan
  2. China
  3. Georgia
  4. India
  5. Indonesia
  6. Japan
  7. Kazakhstan
  8. Korea, Rep.
  9. Malaysia
  10. Pakistan
  11. Philippines
  12. Singapore
  13. Thailand
  14. Viet Nam

Middle East (7):
  1. Bahrain
  2. Israel
  3. Jordan
  4. Qatar
  5. Saudi Arabia
  6. Turkiye
  7. United Arab Emirates

Africa (11):
  1. Benin
  2. Egypt, Arab Rep.
  3. Gabon
  4. Ghana
  5. Mauritius
  6. Morocco
  7. Rwanda
  8. South Africa
  9. Tanzania

In [47]:
for income in perfect_score_2024['income_level'].unique():
    income_count = len(perfect_score_2024[perfect_score_2024['income_level'] == income])
    print(f"{income}: {income_count} countries")


Upper Middle Income: 20 countries
High Income: 38 countries
Lower Middle Income: 10 countries
Low Income: 2 countries


In [48]:
bottom_20 = df_2024.nsmallest(20, 'cybercrime_regulation_score')[['country_name', 'cybercrime_regulation_score', 'region', 'income_level']]


In [49]:
display(bottom_20)


,country_name,cybercrime_regulation_score,region,income_level
109,Eritrea,0.0000,Africa,Low Income
141,Guinea-Bissau,0.0000,Africa,Low Income
181,"Korea, Dem. People's Rep.",0.0000,Asia,Low Income
227,"Micronesia, Fed. Sts.",0.0000,Oceania,Lower Middle Income
219,Marshall Islands,0.0537,Oceania,Upper Middle Income
311,Solomon Islands,0.0537,Oceania,Lower Middle Income
1,Afghanistan,0.1047,Asia,Low Income
359,Tuvalu,0.1047,Oceania,Upper Middle Income
349,Tonga,0.1280,Oceania,Upper Middle Income
55,Burundi,0.1311,Africa,Low Income


In [50]:
fig.add_trace(go.Bar(
    y=bottom_20['country_name'][::-1],
    x=bottom_20['cybercrime_regulation_score'][::-1],
    orientation='h',
    marker_color=COLORS['danger'],
    text=bottom_20['cybercrime_regulation_score'][::-1].round(3),
    texttemplate='%{text:.3f}',
    textposition='outside'
))


In [51]:
fig.update_layout(
    title='<b>Bottom 20 Countries - Cybercrime Regulation Score (2024)</b>',
    xaxis_title='<b>Regulation Score</b>',
    template='plotly_white',
    height=600,
    xaxis=dict(range=[0, max(bottom_20['cybercrime_regulation_score']) + 0.1])
)


In [52]:
from scipy import stats

# Prepare data for ANOVA
income_groups = []
for income in income_order[:4]:
    group_data = df_2024[df_2024['income_level'] == income]['cybercrime_regulation_score'].values
    income_groups.append(group_data)

# One-way ANOVA
f_statistic, p_value = stats.f_oneway(*income_groups)

In [53]:
correlation = scatter_data[2020].corr(scatter_data[2024])


In [54]:
print(f"Pearson correlation coefficient: {correlation:.4f}")
print(f"Interpretation: {'Very strong' if abs(correlation) > 0.9 else 'Strong' if abs(correlation) > 0.7 else 'Moderate' if abs(correlation) > 0.4 else 'Weak'} positive correlation")


Pearson correlation coefficient: 0.8311
Interpretation: Strong positive correlation


In [55]:
regional_yearly = df.groupby(['region', 'year'])['cybercrime_regulation_score'].mean().unstack()


In [56]:
fig = go.Figure(data=go.Heatmap(
    z=regional_yearly.values,
    x=regional_yearly.columns,
    y=regional_yearly.index,
    colorscale='RdYlGn',
    text=regional_yearly.values.round(3),
    texttemplate='%{text:.3f}',
    textfont={"size": 12},
    colorbar=dict(title="Avg Score")
))


In [57]:
fig.update_layout(
    title='<b>Regional Performance Heatmap: Average Scores by Year</b>',
    xaxis_title='<b>Year</b>',
    yaxis_title='<b>Region</b>',
    template='plotly_white',
    height=400
)


In [58]:
for region in regional_yearly.index:
    change = regional_yearly.loc[region, 2024] - regional_yearly.loc[region, 2020]
    print(f"{region}: {change:+.4f} ({change/regional_yearly.loc[region, 2020]*100:+.1f}%)")


Africa: +0.2072 (+41.3%)
Asia: +0.1127 (+17.5%)
Europe: +0.0203 (+2.1%)
Middle East: +0.0634 (+8.7%)
North America: +0.1480 (+28.8%)
Oceania: +0.2059 (+95.5%)
Other: +0.2062 (+65.0%)
South America: +0.2000 (+33.4%)


In [59]:
fig = px.scatter(
    df_2024,
    x='region',
    y='cybercrime_regulation_score',
    color='income_level',
    size='cybercrime_regulation_score',
    hover_name='country_name',
    hover_data={
        'cybercrime_regulation_score': ':.3f',
        'region': True,
        'income_level': True
    },
    title='<b>Interactive Bubble Chart: Region vs Score (colored by Income Level) - 2024</b>',
    labels={'cybercrime_regulation_score': 'Regulation Score'},
    color_discrete_map={
        'High Income': COLORS['secondary'],
        'Upper Middle Income': COLORS['accent'],
        'Lower Middle Income': '#FF7043',
        'Low Income': COLORS['danger'],
        'Not Classified': '#9E9E9E'
    },
    category_orders={'income_level': income_order}
)


In [60]:
avg_2020 = df_2020['cybercrime_regulation_score'].mean()
avg_2024 = df_2024['cybercrime_regulation_score'].mean()
improvement_pct = ((avg_2024 - avg_2020) / avg_2020) * 100


In [61]:
best_region = region_stats.iloc[0]


In [62]:
high_income_avg = df_2024[df_2024['income_level'] == 'High Income']['cybercrime_regulation_score'].mean()
low_income_avg = df_2024[df_2024['income_level'] == 'Low Income']['cybercrime_regulation_score'].mean()


In [63]:
bottom_10_pct = df_2024['cybercrime_regulation_score'].quantile(0.1)
countries_below_threshold = len(df_2024[df_2024['cybercrime_regulation_score'] < 0.5])


In [64]:
stable_countries = len(scatter_data[abs(scatter_data[2024] - scatter_data[2020]) < 0.01])


In [65]:
completeness_score = 100 - (df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100)


In [66]:

income_order = ['High Income', 'Upper Middle Income', 'Lower Middle Income', 'Low Income', 'Not Classified']

scatter_data = df.pivot(index='country_name', columns='year', values='cybercrime_regulation_score').reset_index()
scatter_data = scatter_data.merge(df_2024[['country_name', 'region', 'income_level']], on='country_name')

scatter_data['score_change'] = scatter_data[2024] - scatter_data[2020]

# Top 10 improvers
top_improvers = scatter_data.nlargest(10, 'score_change')[['country_name', 'score_change']]

top_decliners = scatter_data.nsmallest(10, 'score_change')[['country_name', 'score_change']]


## Thank you...pls upvote!!!!